In [ ]:
!python -m spacy download en_core_web_lg
# spacy.load('en_core_web_lg')

In [ ]:
!pip install 'spacy[transformers]'

In [58]:
import pandas as pd
import requests

# URL of the data file
url = "https://groups.csail.mit.edu/sls/downloads/restaurant/restauranttrain.bio"

# Send a GET request to the URL to retrieve the data
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Define an empty list to store the parsed data
    data = []

    # Read the provided data line by line
    lines = response.text.split('\n')
    text = ""
    entity = ""
    current_sentence = []
    current_entities = []
    for line in lines:
        line = line.strip()
        # If the line is empty, it indicates the end of a sentence
        if not line:
            # change this a bit

            # Append the current sentence to the data list
            data.append([text, current_entities])
            # Reset the current_sentence list
            text = ""
            current_entities = []
        else:
            # Split the line into tokens and their corresponding entity labels
            tokens = line.split()
            # print('tokens', tokens)
            start = 0
            if len(text) == 0:
              start = 0
            else:
              text += " "
              start = len(text)
            text += tokens[1]
            end = len(text) - 1
            current_entities.append({"start":start, "end":end + 1, "tag_name":tokens[0].replace("-", "_").upper(), "text": tokens[1]})
            # Append the tokens to the current_sentence list
            # current_sentence.append(tokens)

    # Convert the parsed data into a DataFrame
    df = pd.DataFrame(data, columns=['Text', 'Annotations'])

    # Display the DataFrame
    print(df.head())  # Print the first few rows of the DataFrame
else:
    print("Failed to retrieve data from the URL.")


                                                     Text  \
0                  2 start restaurants with inside dining   
1                                                      34   
2                            5 star resturants in my town   
3               98 hong kong restaurant reasonable prices   
4  a great lunch spot but open till 2 a m passims kitchen   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

In [47]:
pd.set_option('display.max_colwidth', None)

print(df.iloc[0])

Text                                                                                                                                                                                                                                                                                                                                                                         2 start restaurants with inside dining
Annotations    [{'start': 0, 'end': 0, 'tag_name': 'B_RATING', 'text': '2'}, {'start': 2, 'end': 6, 'tag_name': 'I_RATING', 'text': 'start'}, {'start': 8, 'end': 18, 'tag_name': 'O', 'text': 'restaurants'}, {'start': 20, 'end': 23, 'tag_name': 'O', 'text': 'with'}, {'start': 25, 'end': 30, 'tag_name': 'B_AMENITY', 'text': 'inside'}, {'start': 32, 'end': 37, 'tag_name': 'I_AMENITY', 'text': 'dining'}]
Name: 0, dtype: object


In [48]:
df.iloc[2]

Text                                                                                                                                                                                                                                                                                                                                                                                    5 star resturants in my town
Annotations    [{'start': 0, 'end': 0, 'tag_name': 'B_RATING', 'text': '5'}, {'start': 2, 'end': 5, 'tag_name': 'I_RATING', 'text': 'star'}, {'start': 7, 'end': 16, 'tag_name': 'O', 'text': 'resturants'}, {'start': 18, 'end': 19, 'tag_name': 'B_LOCATION', 'text': 'in'}, {'start': 21, 'end': 22, 'tag_name': 'I_LOCATION', 'text': 'my'}, {'start': 24, 'end': 27, 'tag_name': 'I_LOCATION', 'text': 'town'}]
Name: 2, dtype: object

In [59]:
train_dataset = []

for entry in data:
  if len(entry[1]) > 0:
    current_entry = {}
    current_entry['text'] = entry[0]
    current_entry['entities'] = []
    for entity in entry[1]:
      start_index = entity['start']
      end_index = entity['end']
      label = entity['tag_name']
      current_entry['entities'].append((start_index, end_index, label))
    train_dataset.append(current_entry)

In [ ]:
train_dataset

In [53]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm


nlp = spacy.blank("en")
doc_bin = DocBin()


In [61]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
from spacy.util import filter_spans

nlp = spacy.blank("en")
doc_bin = DocBin()

for data in tqdm(train_dataset):
  sentence = data['text']
  # print(sentence)
  tags = data['entities']
  # print(tags)
  # break
  doc = nlp.make_doc(sentence)
  ents = []
  for start_index, end_index, tag in tags:
    # print(start_index, end_index, tag)
    span = doc.char_span(start_index, end_index, label=tag, alignment_mode='contract')
    if span is None:
      continue
    else:
      ents.append(span)
  filtered_entries = filter_spans(ents)
  doc.ents = ents
  # print(doc.ents)
  doc_bin.add(doc)
print(doc_bin)

doc_bin.to_disk('train.spacy')

100%|██████████| 7660/7660 [00:01<00:00, 4217.40it/s]


In [37]:
print(doc_bin)

In [ ]:
# create a config file using spacy.io/usage/training#quickstart

In [65]:
!python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [66]:
!python -m spacy train config.cfg --output ./ --paths.train ./train.spacy --paths.dev ./train.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     90.69   60.49   60.18   60.81    0.60
  0     200         73.56   6510.93   87.53   87.53   87.53    0.88
  0     400        102.71   5044.54   89.64   89.67   89.61    0.90
  1     600        109.27   5375.84   91.61   91.61   91.62    0.92
  1     800        132.08   5743.17   92.41   92.42   92.41    0.92
  2    1000        151.96   6326.63   93.36   93.35   93.36    0.93
  3    1200        182.97   6696.61   94.57   94.57   94.57    0.95
  4    1400        285.36   7169.89   95.34   95.34   95.35    0.95
  5    1600        293.85   7406.62 

In [69]:
spacy_model = spacy.load("model-best")

In [30]:
spacy_model

In [80]:
doc = spacy_model('3 star resturants in my country')

In [82]:
doc.ents

(3, star, resturants, in, my, country)

In [81]:
spacy.displacy.render(doc, style="ent", jupyter=True)